In [1]:
import cv2
import pandas as pd
import time
import numpy as np
from scipy.spatial.distance import euclidean

# Charger la vidéo originale
video_path = "Puente_guambra2.mp4"

# Charger les données du fichier CSV contenant les détections
data = pd.read_csv(f"{video_path}_positions.csv")

data

,frame_number,id,class,conf,x,y,w,h,r
0,1,1,small vehicle,0.802636,3148.414062,1927.137207,64.585747,30.545033,0.689725
1,1,2,small vehicle,0.795741,3054.048584,463.554932,71.559853,32.174553,2.819362
2,1,3,large vehicle,0.775441,3079.295166,1646.043091,190.205017,43.182365,0.649792
3,1,4,small vehicle,0.775234,3848.507080,297.390717,67.181137,28.472355,2.855587
4,1,5,small vehicle,0.774091,238.860870,1675.974121,68.550430,31.932125,0.058043
...,...,...,...,...,...,...,...,...,...
682993,2787,257,small vehicle,0.010771,1617.497070,981.535706,58.366802,25.187868,0.528911
682994,2787,258,small vehicle,0.010659,1522.585083,474.937256,87.213348,34.550034,2.684876
682995,2787,259,small vehicle,0.010522,3660.509521,1577.630249,36.204979,23.738024,2.863581
682996,2787,260,small vehicle,0.010248,3945.224609,213.176300,64.669098,28.852522,2.828396


ploting

In [2]:
# Définir une liste de couleurs pour chaque classe
class_colors = {
    'small vehicle': (0, 255, 255),
    'large vehicle': (255, 255, 0),
    # Ajouter d'autres classes et couleurs au besoin
}


def draw_obb(img, cx, cy, w, h, angle, class_name, confidence, colors=class_colors, thickness=2):
    rect = ((cx, cy), (w, h), angle*180/np.pi)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(img, [box], 0, colors[class_name], thickness)
    cv2.putText(img, f"{class_name} ({confidence:.2f})", (int(cx), int(cy) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[class_name], 2)





cap = cv2.VideoCapture(video_path)

# Lire la vidéo frame par frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    # Filtrer les détections pour le frame actuel
    detections_in_frame = data[data['frame_number'] == frame_number]

    # Dessiner les boîtes englobantes sur le frame actuel
    for index, row in detections_in_frame.iterrows():
        x, y, w, h, r = row['x'], row['y'], row['w'], row['h'], row['r']
        class_name, confidence = row['class'], row['conf']
        draw_obb(frame, float(x), float(y), float(w), float(h), float(r), class_name, float(confidence))

    # Afficher le frame avec les boîtes englobantes
    cv2.imshow("YOLOv8 Obb", cv2.resize(frame, (1920, 1080)))
    
    time.sleep(0.1)

    # Attendre pour la prochaine frame ou sortir de la boucle si 'q' est pressé
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Libérer les ressources
cap.release()
cv2.destroyAllWindows()


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/arthur/.local/lib/python3.10/site-packages/cv2/qt/plugins"


KeyboardInterrupt: 

: 

Unique id

In [32]:
# Définir une liste de couleurs pour chaque classe
class_colors = {
    'small vehicle': (0, 255, 255),
    'large vehicle': (255, 255, 0),
    # Ajouter d'autres classes et couleurs au besoin
}

threshold = 10

look_back = 10


def draw_obb(img, cx, cy, w, h, angle, class_name, confidence, id, points, colors=class_colors, thickness=2):
    rect = ((cx, cy), (w, h), angle*180/np.pi)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(img, [box], 0, colors[class_name], thickness)
    cv2.putText(img, f"{class_name} ({confidence:.2f}) id: {id}", (int(cx), int(cy) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[class_name], 2)
    cv2.polylines(img, [points], isClosed=False, color=colors[class_name], thickness=thickness)  



max_id = 1

cap = cv2.VideoCapture(video_path)

last_positions = []

# Lire la vidéo frame par frame
while cap.isOpened():
    
    ret, frame = cap.read()
    if not ret:
        break
    

    frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    # Filtrer les détections pour le frame actuel
    detections_in_frame = data[data['frame_number'] == frame_number]

    # Dessiner les boîtes englobantes sur le frame actuel
    for index, row in detections_in_frame.iterrows():
        cx, cy, w, h, r = row['x'], row['y'], row['w'], row['h'], row['r']
        class_name, confidence = row['class'], row['conf']
        
        
        detection_center = np.array([cx, cy])
        
        id = None
        
        dist = []
        
        ids = []
        
        for last_position in last_positions:
            
            n_diff_frames = frame_number - last_position['frame_number']
            
            if n_diff_frames <= look_back:
            
                dist.append(euclidean(detection_center, last_position['center']))
                
                ids.append(last_position['id'])
                
        if len(dist) > 0:
            min_dist = min(dist)
            id = ids[dist.index(min_dist)]
            
            if min_dist < threshold:
                id = id
            else:
                id = max_id
                max_id += 1        
        else:
            id = max_id
            max_id += 1
            


            
        last_positions.append({'center': detection_center, 'id': id, 'frame_number': frame_number, 'class': class_name, 'conf': confidence, 'x': cx, 'y': cy, 'w': w, 'h': h, 'r': r})

        track =  [(p.get('x'), p.get("y")) for p in last_positions if p.get('id') == id]

        points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
        
        draw_obb(frame, cx, cy, w, h, r, class_name, float(confidence), id, points)

    # Afficher le frame avec les boîtes englobantes
    cv2.imshow("YOLOv8 Obb", cv2.resize(frame, (1920, 1080)))
    
    time.sleep(0.01)

    # Attendre pour la prochaine frame ou sortir de la boucle si 'q' est pressé
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Libérer les ressources
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

Tracking

In [3]:
# Définir une liste de couleurs pour chaque classe
class_colors = {
    'small vehicle': (0, 255, 255),
    'large vehicle': (255, 255, 0),
    # Ajouter d'autres classes et couleurs au besoin
}

threshold = 10

look_back = 10


def draw_obb(img, cx, cy, w, h, angle, class_name, confidence, id, points, colors=class_colors, thickness=2):
    rect = ((cx, cy), (w, h), angle*180/np.pi)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(img, [box], 0, colors[class_name], thickness)
    cv2.putText(img, f"{class_name} ({confidence:.2f}) id: {id}", (int(cx), int(cy) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[class_name], 2)
    cv2.polylines(img, [points], isClosed=False, color=colors[class_name], thickness=thickness)  



max_id = 1

cap = cv2.VideoCapture(video_path)

last_positions = []

# Lire la vidéo frame par frame
while cap.isOpened():
    
    ret, frame = cap.read()
    if not ret:
        break
    

    frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    # Filtrer les détections pour le frame actuel
    detections_in_frame = data[data['frame_number'] == frame_number]

    # Dessiner les boîtes englobantes sur le frame actuel
    for index, row in detections_in_frame.iterrows():
        cx, cy, w, h, r = row['x'], row['y'], row['w'], row['h'], row['r']
        class_name, confidence = row['class'], row['conf']
        
        
        detection_center = np.array([cx, cy])
        
        id = None
        
        dist = []
        
        ids = []
        
        for last_position in last_positions:
            
            n_diff_frames = frame_number - last_position['frame_number']
            
            if n_diff_frames <= look_back:
            
                dist.append(euclidean(detection_center, last_position['center']))
                
                ids.append(last_position['id'])
                
        if len(dist) > 0:
            min_dist = min(dist)
            id = ids[dist.index(min_dist)]
            
            if min_dist < threshold:
                id = id
            else:
                id = max_id
                max_id += 1        
        else:
            id = max_id
            max_id += 1
            


            
        last_positions.append({'center': detection_center, 'id': id, 'frame_number': frame_number, 'class': class_name, 'conf': confidence, 'x': cx, 'y': cy, 'w': w, 'h': h, 'r': r})

        track =  [(p.get('x'), p.get("y")) for p in last_positions if p.get('id') == id]

        points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
        
        draw_obb(frame, cx, cy, w, h, r, class_name, float(confidence), id, points)

    # Afficher le frame avec les boîtes englobantes
    cv2.imshow("YOLOv8 Obb", cv2.resize(frame, (1920, 1080)))
    
    time.sleep(0.01)

    # Attendre pour la prochaine frame ou sortir de la boucle si 'q' est pressé
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Libérer les ressources
cap.release()
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/arthur/.local/lib/python3.10/site-packages/cv2/qt/plugins"


: 